# Capstone Project 1 Data Wrangling

The goal of this notebook is to perform data wrangling steps on the Walmart Trip Type Classification datasets from Kaggle.  There are two files included: the training data and the test data.  The cleaning steps include:

- loadings the files,
- analyzing the data types of the variables,
- and treating missing values.

## Initial Steps
First we perform standard steps to load the .csv files into Python.

In [1]:
# Import the necessary packages

import pandas as pd
import numpy as np

Set the working directory to be the respository containing the data
which will be different for the user than it is for me.

The original data source is:
https://www.kaggle.com/c/walmart-recruiting-trip-type-classification/data

In [2]:
cd ~/Desktop/Springboard/Capstone_1/Original_Data

/Users/nick/Desktop/Springboard/Capstone_1/Original_Data


In [3]:
# Load the data in Python as pandas data frames

train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

# Create a list of the two data frames to be looped over later.
dfs=[train_df,test_df]

## Exploratory Analysis and Data Types
Now that the data is available, we being by observing the different features in each data frame.

In [4]:
# Inspect column names.

for df in dfs:
    print(df.columns,'\n')

Index(['TripType', 'VisitNumber', 'Weekday', 'Upc', 'ScanCount',
       'DepartmentDescription', 'FinelineNumber'],
      dtype='object') 

Index(['VisitNumber', 'Weekday', 'Upc', 'ScanCount', 'DepartmentDescription',
       'FinelineNumber'],
      dtype='object') 



Notice that the training data has one more feature, `Trip Type`, than the test data.  This is the targer feature.

In [5]:
# Change column names to have proper formatting.

col=['trip_type','visit_number','weekday','upc','scan_count',
     'department_description','fineline_number']
train_df.columns=col
test_df.columns=col[1:len(col)]

Now let's inspect the values of the data to get a rough feel for it.

In [6]:
train_df.head()

,trip_type,visit_number,weekday,upc,scan_count,department_description,fineline_number
0,999,5,Friday,6.811315e+10,-1,FINANCIAL SERVICES,1000.0
1,30,7,Friday,6.053882e+10,1,SHOES,8931.0
2,30,7,Friday,7.410811e+09,1,PERSONAL CARE,4504.0
3,26,8,Friday,2.238404e+09,2,PAINT AND ACCESSORIES,3565.0
4,26,8,Friday,2.006614e+09,2,PAINT AND ACCESSORIES,1017.0


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 647054 entries, 0 to 647053
Data columns (total 7 columns):
trip_type                 647054 non-null int64
visit_number              647054 non-null int64
weekday                   647054 non-null object
upc                       642925 non-null float64
scan_count                647054 non-null int64
department_description    645693 non-null object
fineline_number           642925 non-null float64
dtypes: float64(2), int64(3), object(2)
memory usage: 34.6+ MB


In [8]:
for df in dfs:
    print(df.shape)
    print(df.nunique(),'\n')

(647054, 7)
trip_type                    38
visit_number              95674
weekday                       7
upc                       97714
scan_count                   39
department_description       68
fineline_number            5195
dtype: int64 

(653646, 6)
visit_number              95674
weekday                       7
upc                       98147
scan_count                   49
department_description       67
fineline_number            5203
dtype: int64 



It is clear that each feature is a categorical feature except for `scan_count`.  However, none of them are of this data type yet.  Converting them to this type will reduce the amount of memory required to store and later process the files.

However, `visit_number` is not a meaningful category.  That is to say, it is purely a random identification number that is used to group records into a single purchase which which have a single `trip_type` assigned.  Furthermore, because there are so many unique visit numbers, converting them to categorical data types would actually increase memory usage.

The same memory issue is true for the `upc` feature.  We will have to be careful to ensure that any future analysis does not assume a meaningful ordering of these numeric values.

It should be noted that viewing `department_description` as a category ignores the fact that the values for this variable are word descriptors.  In treating them as categories, we are ignoring this information and eliminating the possibility of text analysis that would attempt to find similar `department_description` values. Naively, I do not think such an analysis would yield strong results anyway.

Lastly, as a side note, there are more unique `upc` and `fineline_number` values in the test data than in the training data.  Whatever prediction model is built in the future must be able to make predictions from enough other data for it to be okay for it to see new categories that it was not trained on.  There could potentially be new `department_description` values as well, but I have not checked this.

In [9]:
train_df['trip_type']=train_df['trip_type'].astype('category')

categoricals=['weekday','department_description','fineline_number']

for df in dfs:
    for category in categoricals:
        df[category]=df[category].astype('category')

The one feature that contained numeric information with a meaningful ordering was `scan_count`.  Here, a positive amount indicates what quantity of the product was purchased, whereas a negative amount indicates how much was returned.  Purchasing and returning an item are conceptually different from each other.  For instance, purchasing 1 vs. 3 counts of an item is vastly different from purchasing 1 vs. returning 1, despite both instances having a `scan_count` difference of 2.  For this reason, I have chosen to split this information into two separate features.

In [10]:
for df in dfs:
    df['purchase_count']=df['scan_count'].clip(lower=0)

for df in dfs:
    df['return_count']=((-1)*df['scan_count']).clip(lower=0)

for df in dfs:
    df.drop(columns='scan_count',axis=1,inplace=True)

## Missing Values
When `train_df.info()` was called earlier, it was observed that there were a few missing values.  Let us return to this point to look at it in more detail.

In [11]:
for df in dfs:
    print(np.sum(pd.isnull(df)),'\n')

trip_type                    0
visit_number                 0
weekday                      0
upc                       4129
department_description    1361
fineline_number           4129
purchase_count               0
return_count                 0
dtype: int64 

visit_number                 0
weekday                      0
upc                       3986
department_description    1328
fineline_number           3986
purchase_count               0
return_count                 0
dtype: int64 



Interestingly, in both data sets `upc` and `fineline_number` are missing an equal number of times.

In [12]:
for df in dfs:
    print(np.sum((pd.isnull(df.fineline_number))&(pd.isnull(df.upc))),'\n')

4129 

3986 



Indeed, `upc` is `NaN` if and only if `fineline_number` is `NaN`.

In [13]:
# Department being NaN is a subset of the others being NaN
for df in dfs:
    print(np.sum((pd.isnull(df.department_description))&(pd.isnull(df.upc))),'\n')

1361 

1328 



Furthermore, the records for which `department_description` is `NaN` is a subset of the records for which `upc` and `fineline_number` are `NaN`.

Because so few of the records contain missing values (under 1%), one option would be to drop these records.  However, the fact that there appears to be structure to the missing values and that such values must be dealt with in the test data suggest that this would be a poor decision.  The data cannot be filled with statistical values such as the mean because it is categorical, and it cannot be backfilled because it is not from a time series. One could assign values randomly or use the mode, but there is no evidence yet to indicate that this would be a sensible decision. Instead, the values will be left blank for now.

## Grouping the Data

Ultimately, the goal is to predict the type of trip each individual customer made.  However, a single trip by a single customer corresponds to multiple records in the dataset because each record is a single item from a single trip by a single customer.  Therefore, we need to group the data by `visit_number`.

In [14]:
train_df_grouped=train_df.groupby('visit_number')
test_df_grouped=test_df.groupby('visit_number')

As a quick sanity check to see that this worked, we will compute whether the number of groups is the same as the number of unique `visit_number` values.

In [15]:
len(train_df_grouped.groups.keys())==train_df.nunique()[1]

True

Here is a sample of the format of the new data.

In [16]:
train_df_grouped.get_group(7)

,trip_type,visit_number,weekday,upc,department_description,fineline_number,purchase_count,return_count
1,30,7,Friday,6.053882e+10,SHOES,8931.0,1,0
2,30,7,Friday,7.410811e+09,PERSONAL CARE,4504.0,1,0


The data is now in a suitable state for an exploratory statisitcal and graphical analysis.